<a href="https://colab.research.google.com/github/AishwaryaPonni/Madrix_text_analysis/blob/main/SentimentAnalysis_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import re
import matplotlib.pyplot as plt
import string
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize.treebank import TreebankWordDetokenizer
from collections import Counter
from wordcloud import WordCloud
from nltk.corpus import stopwords
import nltk
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import gensim
from sklearn.model_selection import train_test_split
import spacy
import pickle
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt 
import tensorflow as tf
import keras
import numpy as np
import pandas as pd

In [4]:
train = pd.read_csv('/content/gdrive/MyDrive/IETE text analysis /imdb dataset/cleaned dataset.csv')
train.head()

,Unnamed: 0,review,sentiment
0,0,one reviewer mentioned watching 1 oz episode y...,positive
1,1,wonderful little production filming technique ...,positive
2,2,thought wonderful way spend time hot summer we...,positive
3,3,basically there family little boy jake think t...,negative
4,4,petter matteis love time money visually stunni...,positive


In [5]:
train['sentiment'].unique()

array(['positive', 'negative'], dtype=object)

In [6]:
#removing html
from bs4 import BeautifulSoup
def strip_html(text):
  soup = BeautifulSoup(text, 'html.parser')
  return soup.get_text()

#removing square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#removing special characters
def remove_special_characters(text):
  pattern = r'[^a-zA-z0-9\s]'
  text = re.sub(pattern,'',text)
  return text

#combining into one function
def denoise_text(text):
  text = strip_html(text)
  text = remove_between_square_brackets(text)
  text = remove_special_characters(text)
  text = text.lower() #converting all letters to lowercase
  return text

In [7]:
max_len = max(len(review) for review in train['review'])
print(max_len)

9206


In [8]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop,Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
train['sentiment'].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [11]:
max_words = 5000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train['review'])
sequences = tokenizer.texts_to_sequences(train['review'])
trainData = pad_sequences(sequences, maxlen=max_len)
print(trainData)


[[   0    0    0 ...  784 3679  350]
 [   0    0    0 ... 1754   17  132]
 [   0    0    0 ...   31   13  112]
 ...
 [   0    0    0 ...  264  388 3594]
 [   0    0    0 ... 1844 2350  603]
 [   0    0    0 ...  898  727    1]]


In [12]:
labels = np.array(train['sentiment'])
y = []
for i in range(len(labels)):
    if labels[i] == 'negative':
        y.append(0)
    if labels[i] == 'positive':
        y.append(1)
y = np.array(y)
labels = tf.keras.utils.to_categorical(y, 2, dtype="float32")
del y
print(labels)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


In [13]:
X_train, X_test, y_train, y_test = train_test_split(trainData,labels, random_state=0)
print (len(X_train),len(X_test),len(y_train),len(y_test))

37500 12500 37500 12500


In [14]:
model1 = Sequential()
model1.add(layers.Embedding(max_words, 20))
model1.add(layers.LSTM(15,dropout=0.5))
model1.add(layers.Dense(2,activation='sigmoid'))
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 20)          100000    
_________________________________________________________________
lstm (LSTM)                  (None, 15)                2160      
_________________________________________________________________
dense (Dense)                (None, 2)                 32        
Total params: 102,192
Trainable params: 102,192
Non-trainable params: 0
_________________________________________________________________


In [15]:
print(y_train)
y_train.shape
y_test.shape


[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [0. 1.]
 [1. 0.]
 [0. 1.]]


(12500, 2)

In [16]:
model1.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])
#Implementing model checkpoins to save the best metric and do not lose it on training.
# checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model1.fit(X_train, y_train, epochs=20,validation_data=(X_test, y_test))

Epoch 1/20
1172/1172 [==============================] - 324s 248ms/step - loss: 0.4946 - accuracy: 0.7548 - val_loss: 0.2899 - val_accuracy: 0.8777
Epoch 2/20
1172/1172 [==============================] - 289s 247ms/step - loss: 0.2660 - accuracy: 0.8941 - val_loss: 0.2805 - val_accuracy: 0.8878
Epoch 3/20
1172/1172 [==============================] - 284s 242ms/step - loss: 0.2443 - accuracy: 0.9036 - val_loss: 0.2648 - val_accuracy: 0.8930
Epoch 4/20
1172/1172 [==============================] - 284s 242ms/step - loss: 0.2437 - accuracy: 0.9021 - val_loss: 0.2983 - val_accuracy: 0.8861
Epoch 5/20
1172/1172 [==============================] - 284s 243ms/step - loss: 0.2384 - accuracy: 0.9057 - val_loss: 0.2903 - val_accuracy: 0.8905
Epoch 6/20
1172/1172 [==============================] - 283s 242ms/step - loss: 0.2328 - accuracy: 0.9071 - val_loss: 0.2691 - val_accuracy: 0.8894
Epoch 7/20
1172/1172 [==============================] - 282s 241ms/step - loss: 0.2272 - accuracy: 0.9098 - val_

In [17]:
test_loss, test_acc = model1.evaluate(X_test, y_test, verbose=2)
print('Model accuracy: ',test_acc)

391/391 - 39s - loss: 0.2603 - accuracy: 0.8944
Model accuracy:  0.8944000005722046


In [ ]:
model2 = Sequential()
model2.add(layers.Embedding(max_words, 40, input_length=max_len))
model2.add(layers.Bidirectional(layers.LSTM(20,dropout=0.6)))
model2.add(layers.Dense(2,activation='sigmoid'))
model2.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['accuracy'])
#Implementing model checkpoins to save the best metric and do not lose it on training.
checkpoint2 = ModelCheckpoint("best_model2.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
history = model2.fit(X_train, y_train, epochs=20,validation_data=(X_test, y_test),callbacks=[checkpoint2])

Epoch 1/20
1172/1172 [==============================] - 577s 490ms/step - loss: 0.4865 - accuracy: 0.7578 - val_loss: 0.2815 - val_accuracy: 0.8856

Epoch 00001: val_accuracy improved from -inf to 0.88560, saving model to best_model2.hdf5
Epoch 2/20
1172/1172 [==============================] - 575s 491ms/step - loss: 0.2705 - accuracy: 0.8909 - val_loss: 0.2701 - val_accuracy: 0.8921

Epoch 00002: val_accuracy improved from 0.88560 to 0.89208, saving model to best_model2.hdf5
Epoch 3/20
1172/1172 [==============================] - 575s 491ms/step - loss: 0.2465 - accuracy: 0.9000 - val_loss: 0.2866 - val_accuracy: 0.8823

Epoch 00003: val_accuracy did not improve from 0.89208
Epoch 4/20
1172/1172 [==============================] - 573s 489ms/step - loss: 0.2435 - accuracy: 0.9040 - val_loss: 0.2624 - val_accuracy: 0.8931

Epoch 00004: val_accuracy improved from 0.89208 to 0.89312, saving model to best_model2.hdf5
Epoch 5/20
1172/1172 [==============================] - 572s 488ms/step -